# Pair Programming Métricas

En el pair programming anterior creastéis vuestro primer modelo de machine learning usando la regresion Lineal. Es el momento, que con vuestros datos evaluéis si es bueno haciendo predicciones. Los objetivo de este pairprogramming son:

In [26]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
#  Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
#  Crossvalidation
# ------------------------------------------------------------------------------
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics

In [27]:
#abrimos los csv donde hemos guardado los df de entrenamiento, test y resultados de los residuos:
df_train= pd.read_csv('../datos/datos_train.csv')

df_test = pd.read_csv('../datos/datos_test.csv')

resultados = pd.read_csv('../datos/resultados-RL.csv')

In [28]:
df = pd.read_csv('../datos/diamonds_def.csv')

### 1. Calculéis las métricas para vuestro modelo


In [29]:
#vamos a comprobar primero qué análisis podemos utilizar
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [30]:
X = df.drop("est_carat", axis = 1)
y = df["est_carat"]

In [31]:
X.head()

,Unnamed: 0,est_depth,est_table,est_price,est_lenght_mm,est_width_mm,est_depth_mm,cut_encoded,clarity_encoded,color_encoded
0,1,-1.597233,1.661056,-0.986397,-1.646799,-1.705394,-1.780396,5,3,2
1,2,0.042616,3.519383,-0.986075,-1.503737,-1.498268,-1.780396,4,1,2
2,3,0.534571,0.267310,-0.983820,-1.369617,-1.354180,-1.316852,2,4,2
3,4,1.272504,0.267310,-0.983498,-1.244438,-1.246114,-1.143024,4,2,6
4,5,0.862541,-0.197272,-0.983175,-1.602092,-1.597328,-1.534138,2,3,7


In [32]:
y.head()

0   -1.268160
1   -1.224362
2   -1.092965
3   -1.049167
4   -1.202462
Name: est_carat, dtype: float64

In [33]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [34]:
#creamos el método con la regresión lineal
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [35]:
y_predict_train = lr.predict(x_train) # hacemos las predicciones para el nº de
y_predict_test = lr.predict(x_test)

In [36]:
df_train = pd.DataFrame({'Real': y_train, 'Predicted': y_predict_train, 'Set': ['Train']*len(y_train)})
df_test  = pd.DataFrame({'Real': y_test,  'Predicted': y_predict_test,  'Set': ['Test']*len(y_test)})
resultados = pd.concat([df_train,df_test], axis = 0)
resultados.head()

,Real,Predicted,Set
8407,0.045802,0.238552,Train
47823,-0.633079,-0.557090,Train
49051,-0.479783,-0.363475,Train
26146,1.951048,1.565876,Train
22957,1.994846,1.706551,Train


In [37]:
resultados['residuos'] = resultados['Real'] - resultados['Predicted']
resultados.head()

,Real,Predicted,Set,residuos
8407,0.045802,0.238552,Train,-0.192750
47823,-0.633079,-0.557090,Train,-0.075988
49051,-0.479783,-0.363475,Train,-0.116308
26146,1.951048,1.565876,Train,0.385172
22957,1.994846,1.706551,Train,0.288296


In [38]:
# fig, ax = plt.subplots(2,2,figsize=(20,20))
# # ploteamos los reales vs los predichos
# sns.regplot(data = resultados[resultados['Set'] == "Train"],
#             x = "Real",
#             y = "Predicted",
#             ax = ax[0,0],
#             color = "grey",
#             line_kws = {"color": "red", "alpha": 0.7 })
# sns.regplot(data = resultados[resultados['Set'] == "Test"],
#             x = "Real",
#             y = "Predicted",
#             color = "gray",
#             line_kws = {"color": "red", "alpha": 0.7 },
#             ax = ax[1,0])
# # ploteamos los residuos
# sns.histplot(resultados[resultados['Set'] == "Train"],
#              x="residuos",
#              color ="grey",
#              kde=True,
#              ax = ax[0,1])
# sns.histplot(resultados[resultados['Set'] == "Test"],
#              x="residuos",
#              color = "grey",
#              kde=True,
#              ax = ax[1,1])
# ax[0,0].set_title("Train reales vs predichos", fontsize = 15, fontweight = "bold")
# ax[0,1].set_title("Train residuos", fontsize = 20, fontweight = "bold")
# ax[1,0].set_title("Test reales vs predichos", fontsize = 15, fontweight = "bold")
# # ax[1,1].set_title("Test residuos", fontsize = 20, fontweight = "bold"

In [39]:
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [40]:
cv_scores = cross_validate(
                estimator = LinearRegression(),
                X         = X,
                y         = y,
                scoring   = ('r2', 'neg_root_mean_squared_error'), #los cálculos que nos interesa realizar
                cv        = 10) #nº de veces que realizamos la comprobación

#volcamos a un df para facilitar su legibilidad
cv_scores = pd.DataFrame(cv_scores)
cv_scores #el valor no es realmente negativo, sino positivo

,fit_time,score_time,test_r2,test_neg_root_mean_squared_error
0,0.068991,0.006922,0.842372,-0.181297
1,0.061338,0.013665,0.935319,-0.132183
2,0.057443,0.017843,0.945741,-0.141913
3,0.039232,0.008955,0.923219,-0.219198
4,0.083150,0.017376,0.654636,-0.663287
5,0.071994,0.013189,0.849583,-0.445127
6,0.063934,0.013442,0.046398,-0.124531
7,0.086819,0.006394,0.781240,-0.094829
8,0.080366,0.031400,0.655110,-0.141283
9,0.073257,0.022741,0.532026,-0.223900


In [41]:
print(f'la media del R2 es = {round(cv_scores["test_r2"].mean(),2)}')

#usamos 'abs' para que nos devuelva el valor absoluto, es decir, que no aparezca negativo
print(f'la media del RMSE es = {round(abs(cv_scores["test_neg_root_mean_squared_error"].mean()),2)}')

la media del R2 es = 0.72
la media del RMSE es = 0.24


In [42]:
resultados_metricas = {'MAE': [mean_absolute_error(y_test, y_predict_test), mean_absolute_error(y_train, y_predict_train)],
                'MSE': [mean_squared_error(y_test, y_predict_test), mean_squared_error(y_train, y_predict_train)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_predict_test)), np.sqrt(mean_squared_error(y_train, y_predict_train))],
                'R2':  [r2_score(y_test, y_predict_test), r2_score(y_train, y_predict_train)],
                 "set": ["test", "train"], 
                 "modelo": ["Linear Regresion", "LinearRegression"]}

df_resultados = pd.DataFrame(resultados_metricas)

df_resultados

,MAE,MSE,RMSE,R2,set,modelo
0,0.152142,0.074400,0.272763,0.925262,test,Linear Regresion
1,0.149893,0.071864,0.268074,0.928192,train,LinearRegression


In [43]:
# Para el conjunto de entrenamsiento las méticas han sido
print('Para el conjunto train:---------------')
print('- El valor de r2 score es ',r2_score(y_train,y_predict_train))
print('- MAE =',mean_absolute_error(y_train,y_predict_train))
print('- MSE = ',mean_squared_error(y_train,y_predict_train))
print('- RMSE =  ',np.sqrt(mean_squared_error(y_train,y_predict_train)))
# Para el conjunto de test las métricas han sido
print('Para el conjunto test:---------------')
print('El valor de r2 score es ',r2_score(y_test,y_predict_test))
print('El MAE es',mean_absolute_error(y_test,y_predict_test))
print('El MSE es',mean_squared_error(y_test,y_predict_test))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_test,y_predict_test)))

Para el conjunto train:---------------
- El valor de r2 score es  0.9281921692913474
- MAE = 0.14989254816726008
- MSE =  0.07186376709482628
- RMSE =   0.2680741820743398
Para el conjunto test:---------------
El valor de r2 score es  0.9252621762553574
El MAE es 0.15214177204818377
El MSE es 0.07439975573429805
EL RMSE es  0.27276318617859346


### 2. Discutid los resultados de las métricas y extraed conclusiones.


### 3. Guardad los resultados de las métricas en un csv para usarlo más adelante.

In [44]:
# vamos a guardar este csv para usarlo en próximas lecciones

df_resultados.to_csv("../datos/diamonds_rdos_metricas.csv")